In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()

from scipy import stats
import glob
import re

In [2]:
# download custom functions
import sys
sys.path.append('model_dev_functions')

from data_prep import get_aggregation_from_window, batch_aggregate_pickle, read_multiple_pickles        # prepare dataset for model development
from feature_engineering import add_NDVI, add_vhvv, get_high_corr_cols                                  # custom functions to help feature engineering
from model_development import base_model_pred, model_scores, df_to_arr                                  # model development use
from submission_format import prediction_to_submission_df                                               # change prediiction array of 0 and 1 to csv file suited for submission

Note: This notebook entails the cleaned up process of our journey to discover the final model that achieves 1.0 f1-score. To check out the final model, skip to section - `iteration 4`. 

# Initial Modelling exploration

With the use of cross-validation, several models are explored to extract general principles for model development-

In [3]:
def model_scores(X, y, scoring):

    # Random Forest classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs = -1)
    rf_scores = cross_val_score(rf_model, X, y, cv=5)

    # Logistic Regression classifier
    lr_model = LogisticRegression(random_state=42, n_jobs = -1)
    lr_scores = cross_val_score(lr_model, X, y, cv=5, scoring = scoring)

    # SVM classifier
    svm_model = SVC(kernel='linear', random_state=42)
    svm_scores = cross_val_score(svm_model, X, y, cv=5, scoring = scoring)

    # XGBoost classifier
    xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42, n_jobs = -1)
    xgb_scores = cross_val_score(xgb_model, X, y, cv=5, scoring = scoring)

    # LightGBM classifier
    lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42, n_jobs = -1)
    lgb_scores = cross_val_score(lgb_model, X, y, cv=5, scoring = scoring)

    return rf_scores, lr_scores, svm_scores, xgb_scores, lgb_scores


Several important general principles are extracted from the initial exploration on modelling.  

#### Principle 1: The use of window return better results than a single pixel.

Random samples 20210301 and 20210311 are chosen from the downloaded Sentinel-2 data and stacked together as features and tested against multiple machine learning models. In this case, windows with 5 pixels around each coordinate is averaged to get the aggregated window values representing the whole window. The performance of models are evaluated with by using the aggregated window values and the single pixel values as data input respectively. The models show improved accuracy with the use of aggregated mean value.

Note: the window size of 5 is selected based on the analysis of distances between sampling points, of which we realise that in between points in the same cluster, most have distance of around 50m between points, corresponding to 5-pixels for the the highest spatial resolution of 10m for Sentinel-2 data.

In [4]:
# read data extracted from Sentinel-2 bands on two dates 210301 and 210311
s2_l2a_210301 = pd.read_pickle("../11-datasets/march-S2/LEVEL1_SENTINEL-2-L2A_2021-03-01.pkl")
s2_l2a_210311 = pd.read_pickle("../11-datasets/march-S2/LEVEL1_SENTINEL-2-L2A_2021-03-11.pkl")

# get columns that have just a single pixel values but not windows
def get_all_one_point_bands(full_df, label_col,
                            label_encode={'Rice': 1, 'Non Rice': 0}):
    '''
    all bands without windows retrieved by extracting the pixel value directly
    '''
    X = full_df[['B02', 'B03', 'B04', 'B08', 'B05', 'B06',
                 'B07', 'B11', 'B12', 'B8A', 'SCL', 'B01', 'B09']]
    y = full_df[label_col].map(label_encode)

    return X, y

Xs_0301, y = get_all_one_point_bands(s2_l2a_210301, 'Class of Land')
Xs_0311, y = get_all_one_point_bands(s2_l2a_210301, 'Class of Land')

# concat data from 210301 and 210311
Xs = pd.concat([Xs_0301, Xs_0311], axis=1).values
y = y.values

In [5]:
# get columns with windows of 5*5 around the coordinate
def get_aggregation_from_window(full_df, window_suffix, label_col, suffix_date, agg_method,
                                label_encode={'Rice': 1, 'Non Rice': 0}):
    '''
    aggregate the w5 features to get a single value and return only these features

    parameters:
    full_df = the pickle file read to be processed
    window_suffix = the window name suffix at the columns with windows, e.g. '_w5', '_w10'. Columns with window suffix '_w5' means it has a window of 5*5 pixels.
    label_col = 'Class of Land'
    suffix_date = date added to the processed columns as suffix to prevent column name clash after concatenating multiple dates
    agg_method = aggregation method, e.g. lambda x: x.mean(), mode(), median(), etc.

    return:
    X, y 
    '''

    windowed_columns = [
        band for band in full_df.columns if window_suffix in band]

    X = full_df[windowed_columns].applymap(agg_method)
    X = X.add_suffix(suffix_date)

    if label_col:
        y = full_df[label_col].map(label_encode)
    else:
        y = None

    return X, y

Xw_0301, y = get_aggregation_from_window(s2_l2a_210301, '_w5', 'Class of Land', '_0301', agg_method= lambda x: x.mean())
Xw_0311, y = get_aggregation_from_window(s2_l2a_210311, '_w5', 'Class of Land', '_0311', agg_method= lambda x: x.mean())

# concat data from 210301 and 210331
Xw = pd.concat([Xw_0301, Xw_0311], axis=1).values
y = y.values

Compare F1-score of different models for single pixel values or aggregated pixel values

In [6]:
# f1 score (single pixel values)
rf_base_stack, lr_base_stack, svm_base_stack, xgb_base_stack, lgb_base_stack = model_scores(Xs, y, 'f1')

print('rf_f1 (single pixel values)= ', rf_base_stack, ', mean_f1 ', rf_base_stack.mean())
print('lr_f1 (single pixel values) = ', lr_base_stack, ', mean_f1= ', lr_base_stack.mean())
print('svm_f1 (single pixel values) = ', svm_base_stack, ', mean_f1 = ', svm_base_stack.mean())
print('xgb_f1 (single pixel values) = ', xgb_base_stack, ', mean_f1 = ', xgb_base_stack.mean())
print('lgb_f1 (single pixel values) = ', lgb_base_stack, ', mean_f1 = ', lgb_base_stack.mean())

rf_f1 (single pixel values)=  [0.9        0.85       0.96666667 0.95833333 0.825     ] , mean_f1  0.9
lr_f1 (single pixel values) =  [0.98305085 0.89655172 0.91803279 0.79452055 0.88709677] , mean_f1=  0.8958505361239115
svm_f1 (single pixel values) =  [0.95652174 0.91666667 0.91803279 0.80916031 0.90163934] , mean_f1 =  0.9004041684576309
xgb_f1 (single pixel values) =  [0.928      0.88549618 0.95652174 0.94827586 0.859375  ] , mean_f1 =  0.9155337568811014
lgb_f1 (single pixel values) =  [0.92561983 0.86363636 0.98305085 0.96551724 0.86153846] , mean_f1 =  0.9198725497445015


In [7]:
# f1 score (aggregated pixel values)
rf_mean_stack, lr_mean_stack, svm_mean_stack, xgb_mean_stack, lgb_mean_stack = model_scores(Xw, y, 'f1')

print('rf_f1 (aggregated pixel values) = ', rf_mean_stack, ', mean_f1 = ', rf_mean_stack.mean())
print('lr_f1 (aggregated pixel values) = ', lr_mean_stack, ', mean_f1 = ', lr_mean_stack.mean())
print('svm_f1 (aggregated pixel values) = ', svm_mean_stack, ', mean_f1 = ', svm_mean_stack.mean())
print('xgb_f1 (aggregated pixel values) = ', xgb_mean_stack, ', mean_f1 = ', xgb_mean_stack.mean())
print('lgb_f1 (aggregated pixel values) = ', lgb_mean_stack, ', mean_f1 = ', lgb_mean_stack.mean())

rf_f1 (aggregated pixel values) =  [1.         0.975      0.99166667 1.         0.64166667] , mean_f1 =  0.9216666666666666
lr_f1 (aggregated pixel values) =  [0.98305085 0.92063492 0.99159664 0.93103448 0.89230769] , mean_f1 =  0.9437249163628646
svm_f1 (aggregated pixel values) =  [0.96551724 0.89230769 0.98333333 0.92857143 0.77027027] , mean_f1 =  0.907999993172407
xgb_f1 (aggregated pixel values) =  [1.         1.         1.         0.99159664 0.69822485] , mean_f1 =  0.9379642981452936
lgb_f1 (aggregated pixel values) =  [1.         1.         1.         1.         0.90909091] , mean_f1 =  0.9818181818181818


#### Principle 2: Adding more dates as features will improve model performance (until they overfit) 

Using the same set of machine learning models, adding more dates as features improve the performance of features. This fits our common sense as crops go through multiple cycles throughout the year and more sampled dates result in more useful features informing the prediction model. 

In our case, we study this by setting a set of number of dates sampled to be used as our predictive features, i.e. 1 day, 2 days, 1 month, 3 months, 4 months. We do not use all available dates throughout the year to avoid overfitting as we only have 600 training samples. 

Prepare data

In [8]:
# read data extracted from Sentinel-2 bands on two dates 210301 and 210311
s2_l2a_210301 = pd.read_pickle("../11-datasets/march-S2/LEVEL1_SENTINEL-2-L2A_2021-03-01.pkl")
s2_l2a_210311 = pd.read_pickle("../11-datasets/march-S2/LEVEL1_SENTINEL-2-L2A_2021-03-11.pkl")

# 1 day (data from 210301)
Xw_0301, y = get_aggregation_from_window(s2_l2a_210301, '_w5', 'Class of Land', '_0301', agg_method= lambda x: x.mean())


# 2 days (concat data from 210301 and 210331)
Xw_0311, y = get_aggregation_from_window(s2_l2a_210311, '_w5', 'Class of Land', '_0311', agg_method= lambda x: x.mean())
Xw_2 = pd.concat([Xw_0301, Xw_0311], axis=1).values

# 1 month (march)
march_s2_paths, march_s2_dfs_list = read_multiple_pickles('../11-datasets/march-S2', ['latitude', 'longitude', 'geometry', 'grouping'])     # read_multiple_pickles is custom function to read all pickle files in folder and 
                                                                                                                                            # return a list of dfs needed as features along with ile_paths
m_s2_df, m_s2_df_list  = batch_aggregate_pickle(march_s2_dfs_list, march_s2_paths, '_w5', 'Class of Land', agg_method=lambda x:x.mean())    # aggregate windows to get a single aggregated value for each window for a list of dfs
m_s2_df = m_s2_df.drop('Class of Land', axis=1)

# 3 months (february, august, december), justification elaborated in iteration 1
fad_s2_paths, fad_s2_dfs_list = read_multiple_pickles('../11-datasets/feb_aug_dec-S2', ['latitude', 'longitude', 'geometry', 'grouping'])
fad_s2_df, fad_s2_df_list  = batch_aggregate_pickle(fad_s2_dfs_list, fad_s2_paths, '_w5', 'Class of Land', agg_method=lambda x:x.mean())
fad_s2_df = fad_s2_df.drop('Class of Land', axis=1)

# 4 months (february, march, august, december)
fmad_s2_df = pd.concat([m_s2_df, fad_s2_df], axis=1)


Testing each datasets with our models (random forest, logistic regression, SVMClassifier, XGBoostClassifier, LightGBMClassifier)

In [9]:
# f1 score (1 day)
rf_mean_stack, lr_mean_stack, svm_mean_stack, xgb_mean_stack, lgb_mean_stack = model_scores(Xw_0301, y.values, 'f1')

print('rf_f1 (1 days = ', rf_mean_stack, ', mean_f1 = ', rf_mean_stack.mean())
print('lr_f1 (1 day) = ', lr_mean_stack, ', mean_f1 = ', lr_mean_stack.mean())
print('svm_f1 (1 day) = ', svm_mean_stack, ', mean_f1 = ', svm_mean_stack.mean())
print('xgb_f1 (1 day) = ', xgb_mean_stack, ', mean_f1 = ', xgb_mean_stack.mean())
print('lgb_f1 (1 day) = ', lgb_mean_stack, ', mean_f1 = ', lgb_mean_stack.mean())

rf_f1 (1 days =  [0.95833333 0.875      0.975      0.94166667 0.8       ] , mean_f1 =  0.9099999999999999
lr_f1 (1 day) =  [0.97435897 0.93333333 0.97435897 0.9047619  0.93548387] , mean_f1 =  0.9444594115561857
svm_f1 (1 day) =  [0.94736842 0.896      0.98305085 0.875      0.85925926] , mean_f1 =  0.9121357055539037
xgb_f1 (1 day) =  [0.95934959 0.90076336 0.97435897 0.94827586 0.70658683] , mean_f1 =  0.8978669230099612
lgb_f1 (1 day) =  [1.         0.93442623 0.98305085 0.93913043 0.7195122 ] , mean_f1 =  0.9152239413740768


In [10]:
# f1 score (2 days)
rf_mean_stack, lr_mean_stack, svm_mean_stack, xgb_mean_stack, lgb_mean_stack = model_scores(Xw_2, y.values, 'f1')

print('rf_f1 (2 days) = ', rf_mean_stack, ', mean_f1 = ', rf_mean_stack.mean())
print('lr_f1 (2 days) = ', lr_mean_stack, ', mean_f1 = ', lr_mean_stack.mean())
print('svm_f1 (2 days) = ', svm_mean_stack, ', mean_f1 = ', svm_mean_stack.mean())
print('xgb_f1 (2 days) = ', xgb_mean_stack, ', mean_f1 = ', xgb_mean_stack.mean())
print('lgb_f1 (2 days) = ', lgb_mean_stack, ', mean_f1 = ', lgb_mean_stack.mean())

rf_f1 (2 days) =  [1.         0.975      0.99166667 1.         0.64166667] , mean_f1 =  0.9216666666666666
lr_f1 (2 days) =  [0.98305085 0.92063492 0.99159664 0.93103448 0.89230769] , mean_f1 =  0.9437249163628646
svm_f1 (2 days) =  [0.96551724 0.89230769 0.98333333 0.92857143 0.77027027] , mean_f1 =  0.907999993172407
xgb_f1 (2 days) =  [1.         1.         1.         0.99159664 0.69822485] , mean_f1 =  0.9379642981452936
lgb_f1 (2 days) =  [1.         1.         1.         1.         0.90909091] , mean_f1 =  0.9818181818181818


In [11]:
# f1 score (whole march)
rf_mean_stack, lr_mean_stack, svm_mean_stack, xgb_mean_stack, lgb_mean_stack = model_scores(m_s2_df, y, 'f1')

print('rf_f1 (2 days) = ', rf_mean_stack, ', mean_f1 = ', rf_mean_stack.mean())
print('lr_f1 (2 days) = ', lr_mean_stack, ', mean_f1 = ', lr_mean_stack.mean())
print('svm_f1 (2 days) = ', svm_mean_stack, ', mean_f1 = ', svm_mean_stack.mean())
print('xgb_f1 (2 days) = ', xgb_mean_stack, ', mean_f1 = ', xgb_mean_stack.mean())
print('lgb_f1 (2 days) = ', lgb_mean_stack, ', mean_f1 = ', lgb_mean_stack.mean())

rf_f1 (2 days) =  [1.         1.         1.         1.         0.64166667] , mean_f1 =  0.9283333333333333
lr_f1 (2 days) =  [0.96       0.88721805 0.98305085 0.92857143 0.82014388] , mean_f1 =  0.9157968412067848
svm_f1 (2 days) =  [1.         0.921875   0.99159664 1.         0.77631579] , mean_f1 =  0.9379574856258293
xgb_f1 (2 days) =  [1.         1.         1.         1.         0.70588235] , mean_f1 =  0.9411764705882353
lgb_f1 (2 days) =  [1.         1.         0.98305085 1.         0.90909091] , mean_f1 =  0.9784283513097073


In [12]:
# f1 score (3 months: feb, aug, dec)
rf_mean_stack, lr_mean_stack, svm_mean_stack, xgb_mean_stack, lgb_mean_stack = model_scores(fad_s2_df, y, 'f1')

print('rf_f1 (2 days) = ', rf_mean_stack, ', mean_f1 = ', rf_mean_stack.mean())
print('lr_f1 (2 days) = ', lr_mean_stack, ', mean_f1 = ', lr_mean_stack.mean())
print('svm_f1 (2 days) = ', svm_mean_stack, ', mean_f1 = ', svm_mean_stack.mean())
print('xgb_f1 (2 days) = ', xgb_mean_stack, ', mean_f1 = ', xgb_mean_stack.mean())
print('lgb_f1 (2 days) = ', lgb_mean_stack, ', mean_f1 = ', lgb_mean_stack.mean())

rf_f1 (2 days) =  [1.         1.         1.         1.         0.98333333] , mean_f1 =  0.9966666666666667
lr_f1 (2 days) =  [1. 1. 1. 1. 1.] , mean_f1 =  1.0
svm_f1 (2 days) =  [1.         1.         1.         1.         0.76433121] , mean_f1 =  0.9528662420382166
xgb_f1 (2 days) =  [1.         0.99159664 1.         1.         1.        ] , mean_f1 =  0.9983193277310924
lgb_f1 (2 days) =  [1.         0.99159664 1.         1.         1.        ] , mean_f1 =  0.9983193277310924


In [13]:
# f1 score (3 months: feb, march, aug, dec)
rf_mean_stack, lr_mean_stack, svm_mean_stack, xgb_mean_stack, lgb_mean_stack = model_scores(fmad_s2_df, y, 'f1')

print('rf_f1 (2 days) = ', rf_mean_stack, ', mean_f1 = ', rf_mean_stack.mean())
print('lr_f1 (2 days) = ', lr_mean_stack, ', mean_f1 = ', lr_mean_stack.mean())
print('svm_f1 (2 days) = ', svm_mean_stack, ', mean_f1 = ', svm_mean_stack.mean())
print('xgb_f1 (2 days) = ', xgb_mean_stack, ', mean_f1 = ', xgb_mean_stack.mean())
print('lgb_f1 (2 days) = ', lgb_mean_stack, ', mean_f1 = ', lgb_mean_stack.mean())

rf_f1 (2 days) =  [1.         1.         1.         1.         0.90833333] , mean_f1 =  0.9816666666666667
lr_f1 (2 days) =  [1. 1. 1. 1. 1.] , mean_f1 =  1.0
svm_f1 (2 days) =  [1. 1. 1. 1. 1.] , mean_f1 =  1.0
xgb_f1 (2 days) =  [1.         0.99159664 1.         1.         1.        ] , mean_f1 =  0.9983193277310924
lgb_f1 (2 days) =  [1.         0.99159664 1.         1.         1.        ] , mean_f1 =  0.9983193277310924


Going through the set of tests, we can notice a significant increase in the model accuracy by increasing the number of dates used as features. 

#### Principle 3: For the same input data, different model can show very different prediction. 

This can be observed throughout the explorations above. It shows that model stacking might be able to help to make the model prediction to become more robust. 

#### Other explorations

While there are many other exploration done via cross-validated experiments with training data, but on hindsight, they are omited as they do not provide much value to our actual improvements in test score. For instance: 

- model stacking
- hyperparameter tuning
- feature selection by removing highly correlated values
- generate NDVI for each date
- adding Sentinel-1 data ('vv', 'vh', 'vv/vh')


This is due to the training samples being highly similar with one another and are not random enough, resuling in most of them having a very close to 1.0 f1-score when evaluated via cross-validation. This makes us being unable to see significant improvements or drop in f1-score with cross-validated evaluation. These techniques are further explored with the actual submissions. 

1. Try with model with best performance- random forest (data feb, aug, dec). 

In [14]:
import sys
sys.path.append("../12-python_20230314")

from dataloader import l1_data_loader
import geopandas as gpd
import pystac
import pystac_client
from tqdm import tqdm
from shapely.geometry import Point,box
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import contextily as ctx
from pystac.extensions.eo import EOExtension as eo

# Submissions

## Iteration 1
test accuracy = 0.89, <br>
- Random Forest
- all February, August, December data
- window size of 5*5, aggregated by mean

 We think that the cropping season is where the rice crop differ the most from the other landcover the most. So we use these months - february august december based on the variance analysis on NDVI for rice crop. As a baseline model, this model only include Sentinel-2 data. 

<img src='image_analysis/NDVI variance analysis.png' alt="Alternative text" />

The variance of NDVI between rice and the mean of non-rice by dates is calculated. From this analysis, we found 3 peaks in the variance and choose to select the 3 months corresponding to them, i.e. February, August and December. (See LEVEL-1 data preparation for code.)

#### data preparation

In [15]:
# get data for february, march and december for Sentinel-2 (training data)

# read multiple pickle files for band data corresponding to available dates in february, august and december
fad_s2_paths, fad_s2_dfs_list = read_multiple_pickles('../11-datasets/feb_aug_dec-S2', ['latitude', 'longitude', 'geometry', 'grouping'])

# aggregate all the features with windows 5*5 by mean for the list of dataframes read from the pickle files
fad_s2_df, fad_s2_df_list  = batch_aggregate_pickle(fad_s2_dfs_list, fad_s2_paths, '_w5', 'Class of Land', agg_method=lambda x:x.mean())

X_train1 = fad_s2_df.drop('Class of Land', axis=1)
y_train1 = fad_s2_df['Class of Land']

In [16]:
# get data for february, march and december for Sentinel-2 (coordinates from submission template)
sub2_fad_paths, sub2_fad_df_list = read_multiple_pickles('../11-datasets/SUBMISSION-template_data', 
                                                         ['id', 'latitude', 'longitude', 'geometry', 'target'], filter_condition='-03-')
template_data_s2_df, template_s2_df_list = batch_aggregate_pickle(sub2_fad_df_list, sub2_fad_paths, '_w5', None)

X_pred1 = template_data_s2_df.values

#### Model development and prediction

In [17]:
# train model and get prediction result
model1 = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs = -1)
model1.fit(fad_s2_df.drop('Class of Land', axis = 1), fad_s2_df['Class of Land'])

pred1 = model1.predict(X_pred1)

In [18]:
# convert to dataframe for submission
submission_df1 = prediction_to_submission_df('../submission/challenge_1_submission_template.csv', pred1)
submission_df1

# submission_df1.to_csv("L1_Submission_1.csv", index=False)

,id,target
0,"(10.18019073690894, 105.32022315786804)",Rice
1,"(10.561107033461816, 105.12772097986661)",Rice
2,"(10.623790611954897, 105.13771401411867)",Rice
3,"(10.583364246115156, 105.23946127195805)",Non Rice
4,"(10.20744446668854, 105.26844107128906)",Rice
...,...,...
245,"(10.308283266873062, 105.50872812216863)",Non Rice
246,"(10.582910017285496, 105.23991550078767)",Non Rice
247,"(10.581547330796518, 105.23991550078767)",Non Rice
248,"(10.629241357910818, 105.15315779432643)",Rice


## Iteration 2
test accuracy = 0.86
- all February, August, December data + NDVI
- window size of 5*5, aggregated by mean
- remove highly correlated features
- stack models

 As we realize from our model development exploration that different models lead to different result, stacking models might lead to improved performance by making our result more robust. However, this reduces our performance instead.

#### Data preparation

In [19]:
# get data for february, march and december for Sentinel-2 (training data)

# read multiple pickle files for band data corresponding to available dates in february, august and december
fad_s2_paths, fad_s2_dfs_list = read_multiple_pickles('../11-datasets/feb_aug_dec-S2', ['latitude', 'longitude', 'geometry', 'grouping'])

# aggregate all the features with windows 5*5 by mean for the list of dataframes read from the pickle files
fad_s2_df, fad_s2_df_list  = batch_aggregate_pickle(fad_s2_dfs_list, fad_s2_paths, '_w5', 'Class of Land', agg_method=lambda x:x.mean())

In [20]:
# get data for february, march and december for Sentinel-2 (coordinates from submission template)
sub2_fad_paths, sub2_fad_df_list = read_multiple_pickles('../11-datasets/SUBMISSION-template_data', 
                                                         ['id', 'latitude', 'longitude', 'geometry', 'target'], filter_condition='-03-')
template_data_s2_df, template_s2_df_list = batch_aggregate_pickle(sub2_fad_df_list, sub2_fad_paths, '_w5', None)

#### Data preprocessing 
to add NDVI and remove highly correlated features

In [21]:
def train_2_processing(list_s2_to_add_NDVI, y_in, corr_thresh):
    X2 = add_NDVI(list_s2_to_add_NDVI)
    high_corr_cols = get_high_corr_cols(X2, corr_thresh)
    X2 = X2.drop(high_corr_cols, axis=1)

    y2 = y_in.copy()

    return X2, y2, high_corr_cols

X_train2, y_train2, corr_cols2 = train_2_processing(fad_s2_df_list, fad_s2_df['Class of Land'], corr_thresh = 0.95)


number of high_corr_cols: 70


In [22]:
def pred_2_processing(list_s2_to_add_NDVI, corr_cols):
    X2 = add_NDVI(list_s2_to_add_NDVI)
    X2 = X2.drop(corr_cols, axis=1)

    return X2

X_pred2 = pred_2_processing(template_s2_df_list, corr_cols2)
X_pred2

,AOT_w5_0209,B02_w5_0209,B08_w5_0209,WVP_w5_0209,visual_w5_0209,B11_w5_0209,SCL_w5_0209,B01_w5_0209,B09_w5_0209,NDVI_0,...,SCL_w5_1216,B09_w5_1216,NDVI_6,B02_w5_1226,B08_w5_1226,WVP_w5_1226,visual_w5_1226,SCL_w5_1226,B01_w5_1226,NDVI_7
0,204.0,586.320007,2676.879883,4240.600098,51.160000,1421.520020,4.00,787.280029,3860.560059,-0.686394,...,9.00,12270.919922,-0.069629,789.880005,1139.680054,4347.000000,89.760002,2.28,727.640015,-0.129313
1,204.0,379.640015,5017.359863,4734.040039,30.160000,1784.520020,4.00,373.440002,4630.279785,-0.890205,...,8.80,13114.280273,0.015639,895.440002,1312.160034,4347.000000,98.800003,7.00,871.000000,-0.151119
2,204.0,858.719971,4147.040039,4255.000000,75.639999,2619.479980,4.40,1945.239990,4604.359863,-0.697297,...,9.00,15611.559570,-0.073813,896.840027,1298.880005,4347.000000,83.680000,8.92,949.880005,-0.226979
3,204.0,1200.560059,688.080017,4377.000000,112.040001,689.159973,6.08,2009.199951,2318.120117,0.229456,...,9.00,16110.000000,-0.037297,1811.680054,1415.640015,4347.000000,222.440002,8.64,2333.879883,0.226469
4,204.0,286.239990,4420.959961,4496.520020,34.639999,1483.280029,4.00,208.199997,4497.319824,-0.859015,...,7.20,4733.200195,-0.306312,580.119995,4616.479980,4508.640137,45.119999,4.00,549.080017,-0.826212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,204.0,705.000000,258.440002,4377.000000,90.760002,147.440002,6.00,650.880005,117.680000,0.549442,...,10.00,1142.959961,0.487675,5862.720215,5614.560059,4347.000000,255.000000,9.00,4649.240234,-0.010744
246,204.0,2149.760010,1895.760010,4377.000000,188.600006,1425.160034,7.36,1920.959961,2152.199951,-0.011612,...,9.00,16110.000000,-0.039739,1522.479980,1028.160034,4347.000000,181.600006,8.76,2415.040039,0.269378
247,204.0,2886.239990,2818.959961,4377.000000,229.559998,1935.599976,7.88,1835.000000,2008.359985,-0.064964,...,9.00,16110.000000,-0.029654,1568.079956,1149.000000,4347.000000,160.919998,9.12,1867.319946,0.158007
248,204.0,683.359985,2796.239990,4598.720215,61.919998,1583.920044,4.00,696.799988,2879.560059,-0.644441,...,8.44,10499.919922,-0.003553,990.520020,1584.319946,4347.000000,95.360001,10.00,978.039978,-0.258276


#### Model development and prediction

1st layer

In [23]:
# train base models
models = [
    RandomForestClassifier(random_state = 42, n_estimators=100),
    LogisticRegression(random_state = 42 ),
    #xgb.XGBClassifier(n_estimators=100, random_state=42, n_jobs = -1),
    lgb.LGBMClassifier(random_state = 42, n_estimators=100)
]


def base_model_pred_for_submission(X_train, y_train, X_pred, models):
    '''
    Use each model in the list of models to predict label based on X_pred,
    with each predicted array combined to form a resultant array with the shape of (250, number_of_models)
    '''
    # Generate predictions from base models
    X = []  # This will store the prediction outputs of each model
    y = []  # This will store the true labels
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_pred)
        X.append(y_pred)

    X_2 = np.array(X).T  # Convert to a 2D array, where each row represents a sample and each column represents a model's prediction

    return X_2 

X_pred2_2 = base_model_pred_for_submission(X_train2, y_train2, X_pred2, models)
X_pred2_2.shape

(250, 3)

2nd layer

In [24]:
from sklearn.model_selection import cross_val_predict
def base_model_pred(X_train, y_train, models):
    '''
    Use each model in the list of models to predict label based on X_train,
    with each predicted array combined to form a resultant array with the shape of (600, number_of_models).

    Used to train the 2nd layer of model.
    '''
    # Generate predictions from base models
    X = []  # This will store the prediction outputs of each model
    y = []  # This will store the true labels
    for model in models:
        y_pred = cross_val_predict(model, X_train, y_train, cv=5)
        X.append(y_pred)

    # Convert to a 2D array, where each row represents a sample and each column represents a model's prediction
    X = np.array(X).T
    y = y_train  # Flatten the true labels into a 1D array

    return X, y

# train 2nd layer model
X_train2_2, y_train2_2 = base_model_pred(X_train2, y_train2, models)
models[0].fit(X_train2_2, y_train2_2)
meta_rf = models[0].predict(X_pred2_2) 

# final prediction
pred2 = meta_rf

In [25]:
# load submission
submission_df2 = prediction_to_submission_df('../submission/challenge_1_submission_template.csv', pred2)
submission_df2

,id,target
0,"(10.18019073690894, 105.32022315786804)",Rice
1,"(10.561107033461816, 105.12772097986661)",Rice
2,"(10.623790611954897, 105.13771401411867)",Non Rice
3,"(10.583364246115156, 105.23946127195805)",Non Rice
4,"(10.20744446668854, 105.26844107128906)",Rice
...,...,...
245,"(10.308283266873062, 105.50872812216863)",Non Rice
246,"(10.582910017285496, 105.23991550078767)",Non Rice
247,"(10.581547330796518, 105.23991550078767)",Non Rice
248,"(10.629241357910818, 105.15315779432643)",Rice


In [26]:
# submission_df2.to_csv("L1_Submission_2.csv", index=False)

## iteration 3
test score = 0.94 / 0.95

we tried adding Sentinel-1 data to the model stacking method used in iteration 2, which improves our model performance significantly. F1-score = 0.94 /  0.95 based on the normalization method we use (MinMax and RobustScaler performs better). To help readability, we will only illustrate experimentation with MinMaxScaler. 

- Sentinel-2: Feb, Aug, Dec raw data + NDVI 
- Sentinel-1: Sentinel-1 raw data data + VH / VV
- drop highly correlated features
- normalization (StandardScaler, MinMaxScaler, RobustScaler)
- stack models



#### Data Preparation

training data

In [27]:
# get data for february, august and december for Sentinel-2 (training data)
fad_s2_paths, fad_s2_dfs_list = read_multiple_pickles('../11-datasets/feb_aug_dec-S2', ['latitude', 'longitude', 'geometry', 'grouping'])     # read multiple pickle files for band data corresponding to available dates in february, august and december
fad_s2_df, fad_s2_df_list  = batch_aggregate_pickle(fad_s2_dfs_list, fad_s2_paths, '_w5', 'Class of Land', agg_method=lambda x:x.mean())      # aggregate all the features with windows 5*5 by mean for the list of dataframes read from the pickle files

# get data for february, august and december for Sentinel-1 (training data)
fad_s1_paths, fad_s1_dfs_list = read_multiple_pickles('../11-datasets/sentinel1a_data', 
                                                        ['latitude', 'longitude', 'geometry', 'grouping'], filter_condition='-03-')     # read multiple pickle files for band data corresponding to available dates in february, august and december
fad_s1_df, fad_s1_df_list  = batch_aggregate_pickle(fad_s1_dfs_list, fad_s1_paths, '_w5', 'Class of Land', agg_method=lambda x:x.mean())      # aggregate all the features with windows 5*5 by mean for the list of dataframes read from the pickle files


template data

In [28]:
# get data for february, march and december for Sentinel-2 (coordinates from submission template)
sub2_fmad_paths, sub2_fmad_df_list = read_multiple_pickles('../11-datasets/SUBMISSION-template_data', 
                                                         ['id', 'latitude', 'longitude', 'geometry', 'target'], filter_condition='-03-')
template_data_s2_df, template_s2_df_list = batch_aggregate_pickle(sub2_fmad_df_list, sub2_fmad_paths, '_w5', None)

# get data for february, march and december for Sentinel-1
sub1_fad_paths, sub1_fad_df_list = read_multiple_pickles('../11-datasets/SUBMISSION-template_data/sentinel1a_template_data', 
                                                         ['id', 'latitude', 'longitude', 'geometry', 'target'], filter_condition='-03-')
template_data_s1_df, template_s1_df_list = batch_aggregate_pickle(sub1_fad_df_list, sub1_fad_paths, '_w5', None)

#### model development

In [30]:
def train_3_processing(list_s2_to_add_NDVI, s1_X_in, y_in, corr_thresh):
    X3 = pd.concat([add_NDVI(list_s2_to_add_NDVI), s1_X_in], axis=1)
    high_corr_cols = get_high_corr_cols(X3, corr_thresh)
    X3 = X3.drop(high_corr_cols, axis=1)

    y3 = y_in.copy()

    return X3, y3, high_corr_cols

X_train3_prescaled, y_train3, corr_cols3 =  train_3_processing(fad_s2_df_list, add_vhvv(fad_s1_df_list), fad_s1_df['Class of Land'], corr_thresh = 0.95)

number of high_corr_cols: 70


In [31]:
def pred_3_processing(list_s2_to_add_NDVI, sub1_X_in, corr_cols):
    X3 = pd.concat([add_NDVI(list_s2_to_add_NDVI), sub1_X_in], axis=1)
    X3 = X3.drop(corr_cols, axis=1)

    return X3

X_pred3_prescaled = pred_3_processing(template_s2_df_list, add_vhvv(template_s1_df_list), corr_cols3)

In [32]:
len(X_pred3_prescaled.columns)

111

In [33]:
scale = MinMaxScaler()
scale.fit(X_train3_prescaled)

X_train3= scale.transform(X_train3_prescaled)
X_pred3 = scale.transform(X_pred3_prescaled)
y_train3 = y_train3

#### model development and prediction

In [34]:
# train base models
models = [
    RandomForestClassifier(random_state = 42, n_estimators=100),
    LogisticRegression(random_state = 42 ),
    #xgb.XGBClassifier(n_estimators=100, random_state=42, n_jobs = -1),
    lgb.LGBMClassifier(random_state = 42, n_estimators=100)
]


def base_model_pred_for_submission(X_train, y_train, X_pred, models):
    # Generate predictions from base models
    X = []  # This will store the prediction outputs of each model
    y = []  # This will store the true labels
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_pred)
        X.append(y_pred)

    X = np.array(X).T  # Convert to a 2D array, where each row represents a sample and each column represents a model's prediction
    # y = y_train  # Flatten the true labels into a 1D array

    return X #, y

X_pred3_2 = base_model_pred_for_submission(X_train3, y_train3, X_pred3, models)
X_pred3_2.shape

(250, 3)

In [37]:
X_train3_2, y_train3_2 = base_model_pred(X_train3, y_train3, models)

models[0].fit(X_train3_2, y_train3_2)
meta_rf = models[0].predict(X_pred3_2) 

# models[1].fit(X_train2_2, y_train2_2)
# meta_lr = models[1].predict(X_pred2_2)

# pred2 = (meta_rf + meta_lr) / 2
pred3 = meta_rf

In [38]:
# load submission
submission_df3 = prediction_to_submission_df('../submission/challenge_1_submission_template.csv', pred3)

In [39]:
# submission_df3.to_csv("L1_Submission_3.csv", index=False)

## iteration 4
test_score = 1.0

As we observe that the inclusion of Sentinel-1 data in `iteration 3` significantly improves our prediction accuracy, we assume that cloud is the most important factor in this classification task. We conduct cloud analysis and find out the dates with the least amount of cloud throughout the year. Based on the dates, we extract their corresponding Sentinel-1 and Sentinel-2 data. increased window size to 9*9 to reduce the impact of possible cloud covering the window. 
- cloud15 data
- drop highly correlated features
- normalization (Robust)
- stack models

#### Cloud cover analysis to extract scenes with low cloud coverage

<img src='image_analysis/cloud cover analysis.png' alt="Alternative text" />

As we know that the Sentinel data does not cover all groups of training data in a single scene, we break down the analysis into East scene and West scene. East scene consists of all groups from training data other than the group furthest to the West. 

With this analysis, we can select satellite data from dates with less than 15% cloud cover over our training samples. As East side scenes contains most of the data groups (i.e. 6 of 7 in the east), therefore east side will be use as the primary critiria. 

As we know that the training data is in the same region as the submission template data, we think that it is reasonable to assume that the dates with less cloud cover for training data means the same for the submission template data. 

#### Data Preparation

In [40]:
# Sentinel-2 data (training)
cloud15S2_paths, cloud15S2_list = read_multiple_pickles('../11-datasets/ready_dataset/ready_dataset_cloud15', 
                                                     ['latitude', 'longitude', 'geometry', 'grouping'])

cloud15_s2_df, cloud15_s2_df_list = batch_aggregate_pickle(cloud15S2_list, cloud15S2_paths, '_w9', None, agg_method=lambda x:x.mean())
cloud_15_s2_y = pd.read_pickle(cloud15S2_paths[0])['Class of Land'].map({'Rice': 1, 'Non Rice': 0}).values

# Sentinel-1 data (training)
cloud15S1_paths, cloud15S1_list = read_multiple_pickles('../11-datasets/ready_dataset/ready_dataset_cloud15S1', 
                                                     ['latitude', 'longitude', 'geometry', 'grouping'])

cloud15_s1_df, cloud15_s1_df_list = batch_aggregate_pickle(cloud15S1_list, cloud15S1_paths, '_w9')
cloud_15_s1_y = pd.read_pickle(cloud15S2_paths[0])['Class of Land'].map({'Rice': 1, 'Non Rice': 0}).values

# concat data from sentinel-2 and sentinel-1
cloud15_s1s2 = pd.concat([cloud15_s2_df, cloud15_s1_df.drop('Class of Land', axis=1)], axis=1)
cloud15_s1s2

,AOT_w9_0115,B02_w9_0115,B03_w9_0115,B04_w9_0115,B08_w9_0115,WVP_w9_0115,visual_w9_0115,B05_w9_0115,B06_w9_0115,B07_w9_0115,...,vh_w9_0520,vv_w9_0520,vh_w9_0601,vv_w9_0601,vh_w9_0818,vv_w9_0818,vh_w9_1205,vv_w9_1205,vh_w9_1229,vv_w9_1229
0,138.0,620.629639,967.333313,463.370361,3964.024658,3326.357910,47.592594,1186.308594,3340.926025,4031.987549,...,0.036744,0.157805,0.031072,0.078085,0.007117,0.038904,0.036330,0.177955,0.006644,0.037547
1,138.0,676.679016,904.086426,496.061737,3089.407471,3345.357910,50.901234,1211.271606,2910.061768,3468.333252,...,0.028177,0.349164,0.027931,0.092954,0.008786,0.089624,0.016413,0.055847,0.004707,0.055702
2,138.0,594.518494,899.370361,453.098755,3599.037109,3260.691406,46.592594,1132.012329,2945.061768,3540.209961,...,0.036251,0.199289,0.031167,0.148355,0.006448,0.020774,0.039174,0.153203,0.004183,0.022632
3,138.0,612.617310,969.024719,457.345673,4026.049316,3344.580322,46.987656,1221.802490,3488.481445,4222.728516,...,0.029057,0.136654,0.024184,0.078517,0.010887,0.050735,0.049119,0.186494,0.009598,0.067469
4,138.0,591.839478,699.641968,391.703705,2411.037109,3597.086426,40.271606,754.666687,1883.036987,2293.296387,...,0.032237,0.182050,0.029203,0.071724,0.011458,0.096539,0.008988,0.039027,0.004649,0.032278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,138.0,3547.037109,3536.123535,3399.456787,5190.913574,3244.000000,255.000000,4135.827148,4902.740723,5308.320801,...,0.077351,0.296637,0.089087,0.326931,0.072536,0.369975,0.058265,0.278032,0.052680,0.255063
596,138.0,3677.234619,3715.728516,3639.283936,5169.777832,3244.000000,255.000000,4065.308594,4804.962891,5194.530762,...,0.072449,0.313707,0.087714,0.358288,0.075839,0.311691,0.066887,0.242163,0.068978,0.226601
597,138.0,3955.530762,3922.024658,3810.567871,5253.185059,3244.000000,255.000000,4083.481445,4751.197754,5104.567871,...,0.071250,0.307097,0.079684,0.337885,0.077614,0.263658,0.065535,0.248294,0.080965,0.230987
598,138.0,3752.493896,3682.567871,3530.469238,4916.493652,3244.000000,255.000000,4017.938232,4671.617188,5010.147949,...,0.086430,0.282350,0.074414,0.339348,0.073026,0.258110,0.065576,0.230867,0.078133,0.258629


In [41]:
# Sentinel-2 data (template)
cloud15S2t_paths, cloud15S2t_list = read_multiple_pickles('../11-datasets/ready_dataset/ready_dataset_cloud15t', 
                                                     ['latitude', 'longitude', 'geometry', 'grouping', 'target'])
cloud15_s2t_df, cloud15_s2t_df_list = batch_aggregate_pickle(cloud15S2t_list, cloud15S2t_paths, '_w9', None)


# Sentinel-1 data (template)
cloud15S1t_paths, cloud15S1t_list = read_multiple_pickles('../11-datasets/ready_dataset/ready_dataset_cloud15S1t', 
                                                     ['latitude', 'longitude', 'geometry', 'grouping', 'target'])
cloud15_s1t_df, cloud15_s1t_df_list = batch_aggregate_pickle(cloud15S1t_list, cloud15S1_paths, '_w9', None)

# concat data
cloud15_s1s2t = pd.concat([cloud15_s2t_df, cloud15_s1t_df], axis=1)
X_pred6 = cloud15_s1s2t

In [44]:
# train model
model6 = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs = -1)
model6.fit(cloud15_s1s2, cloud_15_s1_y)

pred6 = model6.predict(X_pred6)

(1.0, 1.0)

In [43]:
submission_df6 = prediction_to_submission_df('../submission/challenge_1_submission_template.csv', pred6)
# submission_df6.to_csv('submission_csv/L1_Submission_6.csv', index=False)

#### Export final model

In [46]:
#import joblib
# joblib.dump(model6, 'RFCloudless_model.h5')

['RFCloudless_model.h5']

#### iteration 4.1 (extra exploration to see if we can get the same accuracy)

The predicted value is the same as the one that has 1.0 f1-score. 

- Add NDVI + VH/VV before feaeture selection

In [ ]:
X_train6_2 = pd.concat([add_NDVI(cloud15_s2_df_list), add_vhvv(cloud15_s1_df_list)], axis=1)
X_pred6_2 = pd.concat([add_NDVI(cloud15_s2t_df_list), add_vhvv(cloud15_s1t_df_list)], axis=1)


high_corr6_2 = get_high_corr_cols(X_train6_2, 0.95)

X_train6_2 = X_train6_2.drop(high_corr6_2, axis=1)
X_pred6_2 = X_pred6_2.drop(high_corr6_2, axis=1)

number of high_corr_cols: 83


In [ ]:
model6.fit(X_train6_2, cloud_15_s1_y)
pred_6_2 = model6.predict(X_pred6_2)

(1.0, 1.0)